In [124]:
import cv2
import numpy as np

def read_image(image_path):
    """Read the image from the specified path."""
    image = cv2.imread(image_path)
    return image

# Define the image path
image_path = "01 - lol easy.jpg"
    
# Step 1: Read the image
image = read_image(image_path)

cv2.imshow("Read the image", image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [125]:
def convert_to_grayscale(image):
    """Convert the image to grayscale."""
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    return gray_image

# Step 2: Convert the image to grayscale
gray_image = convert_to_grayscale(image)

cv2.imshow("Grayscale image", gray_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [126]:
def apply_threshold(gray_image):
    """Apply binary thresholding to highlight edges or areas of high contrast."""
    _, thresh_image = cv2.threshold(gray_image, 200, 255, cv2.THRESH_BINARY_INV)
    return thresh_image

# Step 3: Apply binary thresholding
thresh_image = apply_threshold(gray_image)

cv2.imshow("Binary Thresholding", thresh_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [127]:
def sobel_edge_detection(thresh_image):
    if thresh_image is None:
        raise ValueError("Image not found or unable to read.")

    # Apply Sobel operator
    sobel_x = cv2.Sobel(thresh_image, cv2.CV_64F, 1, 0, ksize=3)  # Sobel in x direction
    sobel_y = cv2.Sobel(thresh_image, cv2.CV_64F, 0, 1, ksize=3)  # Sobel in y direction

    # Calculate the magnitude of the gradients
    sobel_magnitude = np.sqrt(sobel_x**2 + sobel_y**2)
    
    # Convert to uint8
    sobel_magnitude = np.uint8(np.clip(sobel_magnitude, 0, 255))

    return sobel_magnitude

# Step 4: Apply sobel edge detection
sobel_magnitude = sobel_edge_detection(thresh_image)

cv2.imshow("Binary Thresholding", sobel_magnitude)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [128]:
def find_contours(sobel_magnitude):
    contours, hierarchy = cv2.findContours(sobel_magnitude, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    return contours, hierarchy

contours, hierarchy = cv2.findContours(sobel_magnitude, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

# Draw contours on the original image
contour_image = image.copy()
cv2.drawContours(contour_image, contours, -1, (0, 255, 0), 2)  # Draw all contours in green

# Display the original image and the contour image
cv2.imshow('Contours', contour_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [129]:
def is_straight_line(contour):
    # A contour is considered a straight line if it has only two points or if the approximation is close to a line
    epsilon = 0.05 * cv2.arcLength(contour, True)  # Increased epsilon value
    approx = cv2.approxPolyDP(contour, epsilon, True)
    return len(approx) == 2

In [130]:
def approximate_contour(contour, epsilon_factor=0.02):
    """Approximate a contour using the Douglas-Peucker algorithm."""
    # Calculate the perimeter of the contour
    perimeter = cv2.arcLength(contour, True)

    # Calculate epsilon based on the perimeter and the specified factor
    epsilon = epsilon_factor * perimeter

    # Approximate the contour
    approx_contour = cv2.approxPolyDP(contour, epsilon, True)

    return approx_contour

In [131]:
def are_all_contours_straight_lines(contours):
    """Check if all contours are straight lines."""
    return all(is_straight_line(contour) for contour in contours)

In [132]:
# Step 7: Handle contours based on their shape
if are_all_contours_straight_lines(contours):
    print("All contours are straight lines. Proceeding with contour_detection.")
    processed_contours = contours  # You can call your contour_detection function here if needed
else:
    print("Not all contours are straight lines. Proceeding with approximate_contour.")
    processed_contours = [approximate_contour(contour) for contour in contours]

# Final Step: Display the processed contours
for contour in processed_contours:
    cv2.drawContours(image, [contour], -1, (0, 255, 0), 2)

cv2.imshow("Processed Contours", image)
cv2.waitKey(0)
cv2.destroyAllWindows()

Not all contours are straight lines. Proceeding with approximate_contour.


In [133]:
def filter_contours(contours, min_area=100, max_area=10000, min_perimeter=50):
    """Filter contours based on area and perimeter."""
    filtered_contours = []
    for contour in contours:
        area = cv2.contourArea(contour)
        perimeter = cv2.arcLength(contour, True)
        
        if (min_area < area < max_area) and (perimeter > min_perimeter):
            filtered_contours.append(contour)
    return filtered_contours

# Filter the contours based on area and perimeter
filtered_contours = filter_contours(approximated_contours, min_area=100, max_area=5000, min_perimeter=50)
print(f"Number of filtered contours: {len(filtered_contours)}")

cv2.imshow('Filtered Contours', contour_image)
cv2.waitKey(0)
cv2.destroyAllWindows()


Number of filtered contours: 35


In [134]:
def draw_detected_rectangles(image, barcode_rects):
    """Draw the detected rectangles for visualization."""
    for rect in barcode_rects:
        cv2.polylines(image, [rect], isClosed=True, color=(0, 255, 0), thickness=2)
    
    rectangular_image = image
    return rectangular_image

In [135]:
def extract_roi(image, barcode_rects):
    """Extract the region of interest (ROI) to crop the region out for further processing."""
    rois = []
    for rect in barcode_rects:
        x, y, w, h = cv2.boundingRect(rect)
        roi = image[y:y + h, x:x + w]
        rois.append(roi)
    return rois

In [136]:
# Step 5: Approximate contours
barcode_rects = approximate_contours(contours)

cv2.imshow("Approximate Contours", barcode_rects)
cv2.waitKey(0)
cv2.destroyAllWindows()
    
# Step 6: Draw the detected rectangles
image_with_rectangles = draw_detected_rectangles(image, barcode_rects)

cv2.imshow("detected rectangles", image_with_rectangles)
cv2.waitKey(0)
cv2.destroyAllWindows()

error: OpenCV(4.10.0) :-1: error: (-5:Bad argument) in function 'imshow'
> Overload resolution failed:
>  - mat is not a numpy array, neither a scalar
>  - Expected Ptr<cv::cuda::GpuMat> for argument 'mat'
>  - Expected Ptr<cv::UMat> for argument 'mat'


In [ ]:
# Step 7: Extract the region of interest (ROI)
rois = extract_roi(image, barcode_rects)

cv2.imshow("ROI", rois)
cv2.waitKey(0)
cv2.destroyAllWindows()
    
# Display the results
#cv2.imshow("Detected Barcodes", image_with_rectangles)
#for i, roi in enumerate(rois):
#    cv2.imshow(f"ROI {i + 1}", roi)
    
#cv2.waitKey(0)
#cv2.destroyAllWindows()